


First We need to import two python libraries and load the data.

In [1]:
from pulp import *
import pandas as pd

# load the data
df = pd.read_excel('D:/Georgian tech/Courses/ISYE6501/Homeworks/Week 12/diet.xls')

Then we need to create two arrays to store the minimum and maximum demand of each kind of nutrtion.The lines including the demand values are then removed from the main table. The rest part of the main table will be used in the optimization calculation.

In [2]:
# create the maximum demand array 
demand_max=df.iloc[-1].values[3:]

#create the minimum demand array
demand_min=df.iloc[-2].values[3:]

In [3]:
# remove unnessary parts in the table
df.drop(df.index[-3:],inplace=True)

Our next step is to bulid up the optimization model. The objective of the optimization model is to minimize the cost per diet.

In [4]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Diet Problem", LpMinimize)

A LP dictionary variable is created here to contain all decision variables. The data type of decision variable is continous and its lower bound is set to be 0. To facilitate following calculation and presentation of the optimization results, a dataframe is made to hold all decision variables. A list is also created to include the name of all decision variables for further use. 

In [5]:
# Get a list of all food names 
col=df['Foods'].values

# A dictionary called 'ingred_vars' is created to contain the referenced Variables
ingred_vars = LpVariable.dicts("Ingr",df['Foods'].values,0,cat="Continuous")  #notice how to set the catergory
ing=pd.DataFrame([ingred_vars],index=[0],columns=col) #very import to set the column names here, or the order of columns would be different

#create a list of target value names for further usage
col_ing=[]
for i in col:
    col_ing.append(str(ing[i][0]))

Then, we define the objective function, which is the sum of costs of all ingredients used in the diet. 

In [6]:
# The objective function is added to 'prob' first
prob+=lpSum(ing.dot(df['Price/ Serving'].values))     

When the objective function is defined, our next step is to add necessary constraints to the model. In part 1 of the question,we only need to make sure the minimum and maximum nutritional demand of each kind are met.

In [7]:
# make a list of constraints
constraints= df.columns[3:]  

# Add the expression of all constraints by a for loop
for i in range(len(constraints)):
    prob+=lpSum(ing.dot(df[constraints[i]].values))>=demand_min[i]
    prob+=lpSum(ing.dot(df[constraints[i]].values))<=demand_max[i]             

Then we can run the model to solve the problem.The result of the solving process as well as the optimal total cost per meal are shown as follows. 

In [8]:
# The problem data is written to an .lp file
prob.writeLP("Diet.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per meal = ", round(pulp.value(prob.objective),2))

Status: Optimal
Total Cost of Ingredients per meal =  4.34


What we want to know from the result is which foods are selected in the meal and whether the diet plan meet the nutritional demand or not. We can answer these questions by the following codes. 

In [9]:
# Each of the variables is stored in a dict varibale with it's resolved optimum value
result={}
for v in prob.variables():
    result[v.name]=v.varValue

# The result is converted into a dataframe for illustration
Ing_result=pd.DataFrame(result,index=['Serv_quant'],columns=col_ing)
optim=Ing_result.values[0] # get the optimum values

# The food included in the diet is shown as follows in the order of its quantity
Ing_result[Ing_result.columns[Ing_result.loc['Serv_quant']>0]].sort_values(by='Serv_quant', axis=1, ascending=False)

,"Ingr_Lettuce,Iceberg,Raw","Ingr_Celery,_Raw","Ingr_Popcorn,Air_Popped",Ingr_Oranges,Ingr_Frozen_Broccoli,Ingr_Poached_Eggs
Serv_quant,63.988506,52.64371,13.869322,2.292939,0.259607,0.141844


The foods used in the diet and their quantities are shown in the table above. 

In [10]:

#check the supply of nutritions per meal
final=[]
for i in range(len(constraints)):
    final.append(round(optim.dot(df[constraints[i]].values),2))
    
#Compare our result with the requirements
Compare_all=pd.DataFrame([demand_min,demand_max,final],index=['Demand_min','Demand_max','Actual_supply'],columns=constraints)
Compare_all

,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
Demand_min,1500.00,30.0,20.00,800.0,130.0,125.0,60.00,1000.0,400.0,700.00,10.00
Demand_max,2500.00,240.0,70.00,2000.0,450.0,250.0,100.00,10000.0,5000.0,1500.00,40.00
Actual_supply,2176.31,30.0,23.28,2000.0,450.0,125.0,81.45,10000.0,400.0,1289.19,28.95


We can see from the table above that the diet plan made by the optimization model meets all nutritional requirements. 

### Question 15.2 Part 2

we need to add three additional constraints to our model based on the conditions provided by the question. Another LP dict variable is created to indicated whether each kind of food is used in the diet. This variable is of the binary type, value '1' indicating that it is used in the diet, while value '0' means it is not selected. 

The first additional constraint is added in the following way. If one food is selected, then its serving unit in the diet should be greater than 0.1. And its maximum supply of Calories should be no more than the maximum calory demand in the diet. 

In [11]:
# Create a variable to show whether one ingredient is selected
stat_vars = LpVariable.dicts("Stat",df['Foods'].values,0,1,cat='Binary')  #notice how to set the catergory

# Add more constraints
for i in range(len(col)):
    prob+=ingred_vars[col[i]]>=0.1*stat_vars[col[i]]
    prob+=ingred_vars[col[i]]*df['Calories'].iloc[i]<=2500*stat_vars[col[i]]

Since frozen Broccoli and raw celery should not be used together in the diet, the sum of their status values should be no more than 1. Therefore, the second additional constaint is expressed as follows:

In [12]:
#Constraint of Celery and Frozen Brocconi
prob+=stat_vars['Frozen Broccoli']+stat_vars['Celery, Raw']<=1

To add the third constraint, we need to create a list of foods that could be taken as meat, poultry, fish or egg. 

In [13]:
Proteins=['Roasted Chicken','Poached Eggs','Scrambled Eggs','Bologna,Turkey','White Tuna in Water',
          'Frankfurter, Beef','Ham,Sliced,Extralean','Kielbasa,Prk','Hotdog, Plain','Pork','Sardines in Oil',
           'Splt Pea&Hamsoup','Vegetbeef Soup','Beanbacn Soup,W/Watr']

Then, the third constraint is defined as follows:

In [14]:
#Constraint of proteins
#this is the sum of all statue varibles of proteins
Stat_Proteins=sum([stat_vars[i] for i in Proteins])

#make sure the sum is no less than 3 times of each protein variable
for i in Proteins:
    prob+=Stat_Proteins>=3*stat_vars[i]

Then we can solve the optimization model again with these additinal constraints.

In [15]:
# The problem data is written to an .lp file
prob.writeLP("Diet.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# The optimised objective function value is printed to the screen
print("Total Cost of Ingredients per meal = ", round(pulp.value(prob.objective),2))

Status: Optimal
Total Cost of Ingredients per meal =  4.51


In [16]:
# Each of the variables is stored in a dict varibale with it's resolved optimum value
result={}
for v in prob.variables():
    result[v.name]=v.varValue

# The result is converted into a dataframe for illustration
Ing_result=pd.DataFrame(result,index=['Serv_quant'],columns=col_ing)
optim=Ing_result.values[0] # get the optimum values

# The food included in the diet is shown as follows in the order of its quantity
Ing_result[Ing_result.columns[Ing_result.loc['Serv_quant']>0]].sort_values(by='Serv_quant', axis=1, ascending=False)

,"Ingr_Lettuce,Iceberg,Raw","Ingr_Celery,_Raw","Ingr_Popcorn,Air_Popped",Ingr_Oranges,Ingr_Peanut_Butter,Ingr_Poached_Eggs,Ingr_Scrambled_Eggs,"Ingr_Kielbasa,Prk"
Serv_quant,82.802586,42.399358,13.223294,3.077184,1.942972,0.1,0.1,0.1


The foods used in the diet and their quantities are shown in the table above. We can see that when putting three additional constraints, the diet plan changes accordingly. People don't need to eat frozen broccoli and celery together, they also can get more eggs and a tiny amount of pork.We can see that the minimum serving unit is 0.1, as is required. Hope they like our improved diet plan.

Let's check the nutrition supply of this improved the plan again.

In [17]:
#check the supply of nutritions per meal
final=[]
for i in range(len(constraints)):
    final.append(round(optim.dot(df[constraints[i]].values),2))
    
#Compare our result with the requirements
Compare_all=pd.DataFrame([demand_min,demand_max,final],index=['Demand_min','Demand_max','Actual_supply'],columns=constraints)
Compare_all

,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
Demand_min,1500.0,30.00,20.00,800.0,130.0,125.0,60.00,1000.00,400.0,700.00,10.00
Demand_max,2500.0,240.00,70.00,2000.0,450.0,250.0,100.00,10000.00,5000.0,1500.00,40.00
Actual_supply,2500.0,44.01,53.75,2000.0,450.0,125.0,94.52,9371.84,400.0,1224.61,28.99


We can see that all nutritional requirements are met in the improved diet plan. During the process of building up the optimization model, I used Excel solver to build a similar model to do the crossing validation work, it is very helpful. 